In [1]:
!pip install -q langchain-community==0.2.0 sentence-transformers faiss-cpu pypdf


  You can safely remove it manually.


In [2]:
#load & split
from pathlib import Path
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

pdf_dir = Path("../data/raw/hci_papers")
docs = []
for pdf in pdf_dir.glob("*.pdf"):
    docs.extend(PyPDFLoader(str(pdf)).load())

splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=150)
chunks = splitter.split_documents(docs)
print("Chunks:", len(chunks))


Chunks: 250


In [ ]:
# embed & save FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

embedder = HuggingFaceEmbeddings(
    model_name="intfloat/e5-mistral-7b-instruct",
    encode_kwargs={"normalize_embeddings": True}
)

vectordb = FAISS.from_documents(chunks, embedder)
vectordb.save_local("../data/processed/faiss_hci")
print("✔ FAISS index saved to data/processed/faiss_hci")


C:\Users\Akhan\anaconda3\envs\rag_hci\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\Akhan\anaconda3\envs\rag_hci\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Akhan\.cache\huggingface\hub\models--intfloat--e5-mistral-7b-instruct. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to

In [1]:
!pip install -q langchain-community==0.2.0 sentence-transformers faiss-cpu pypdf


In [2]:
from pathlib import Path
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [2]:
from pathlib import Path
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

pdf_dir = Path("../data/raw/hci_papers")
docs = []
for pdf in pdf_dir.glob("*.pdf"):
    docs.extend(PyPDFLoader(str(pdf)).load())

splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=150)
chunks = splitter.split_documents(docs)

# --- extract plain strings & metadata ---
texts      = [doc.page_content for doc in chunks]
metadatas  = [doc.metadata     for doc in chunks]

print(f"Loaded {len(texts)} chunks from {len(list(pdf_dir.glob('*.pdf')))} PDFs")


Loaded 653 chunks from 8 PDFs


In [6]:
pip install hf_xet

Note: you may need to restart the kernel to use updated packages.


In [5]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from tqdm.auto import tqdm

EMB_MODEL = "BAAI/bge-small-en-v1.5"               # or MiniLM / e5-small
embedder  = HuggingFaceEmbeddings(
    model_name=EMB_MODEL,
    encode_kwargs={"normalize_embeddings": True},
)

good_vecs, good_texts, good_meta = [], [], []

for txt, meta in tqdm(zip(texts, metadatas), total=len(texts), desc="Embedding"):
    try:
        vec = embedder.embed_query(str(txt))       # encodes ONE chunk
        good_vecs.append(vec)
        good_texts.append(str(txt))
        good_meta.append(meta)
    except Exception as e:
        # If one chunk is malformed, skip & continue
        continue

print(f"✓ embedded {len(good_texts)} / {len(texts)} chunks")

vectordb = FAISS.from_embeddings(good_vecs, good_texts, good_meta)
vectordb.save_local("../data/processed/faiss_hci_small")
print("✔ FAISS index saved")


Embedding: 100%|█████████████████████████████████████████████████████████████████████| 653/653 [01:41<00:00,  6.42it/s]

✓ embedded 652 / 653 chunks


ValueError: too many values to unpack (expected 2)

In [ ]:
# sanity‑check retrieval latency
import time
retriever = vectordb.as_retriever(search_kwargs={"k": 4})
q = "How should designers incorporate user feedback when building GenAI features?"
t0 = time.time()
docs = retriever.get_relevant_documents(q)
print("First snippet:", docs[0].page_content[:250], "…")
print("Latency:", round((time.time()-t0)*1000), "ms")


In [12]:
#Confirm retrieval works
retriever = vectordb.as_retriever(search_kwargs={"k":3})
print(retriever.get_relevant_documents(
      "How do designers incorporate user feedback when building GenAI features?")[0].page_content[:500])


importance than the generated artifacts themselves. 
Weisz et al. 
that involve trade-ofs between model capabilities and user 
needs, motivated by work that focuses simultaneously on 
end-users of systems [55] and on designers as strategic and 
collaborative end-users of guidelines [82, 87]; and 
• Sensitize designers to the possible risks of generative AI 
applications and their potential to cause a variety of harms 
(inadvertent or intentional), and outline processes that could 
be used to avo


In [15]:
# Cell in 02_ingest_pdfs.ipynb

!dvc add ../data/processed/faiss_hci_small
!git add ../data/processed/faiss_hci_small.dvc notebooks/02_ingest_pdfs.ipynb
!git commit -m "feat: notebook 02 – FAISS index (cleaned) with MiniLM"
!dvc push
!git push



To track the changes with git, run:

	git add '..\data\processed\faiss_hci_small.dvc' '..\data\processed\.gitignore'

To enable auto staging, run:

	dvc config core.autostage true


\u280b Checking graph

fatal: pathspec 'notebooks/02_ingest_pdfs.ipynb' did not match any files


On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   Notebook_01.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	../Untitled.ipynb
	../data/processed/.gitignore
	../data/processed/faiss_hci_small.dvc
	../data/raw/.gitignore
	02_ingest_pdfs.ipynb
	Untitled.ipynb

no changes added to commit (use "git add" and/or "git commit -a")


[WARNING] Unstaged files detected.
[INFO] Stashing unstaged files to C:\Users\Akhan\.cache\pre-commit\patch1746519708-18012.
black................................................(no files to check)Skipped
ruff.................................................(no files to check)Skipped
fix end of files.....................................(no files to check)Skipped
trim trailing whitespace.............................(no files to check)Skipped
[INFO] Restored changes from C:\Users\Akhan\.cache\pre-commit\patch1746519708-18012.


3 files pushed


Everything up-to-date


In [17]:
!git add ../data/processed/faiss_hci_small.dvc 02_ingest_pdfs.ipynb


In [18]:
!git commit -m "feat: notebook 02 – FAISS index (cleaned) with MiniLM"


[main 8d3ac95] feat: notebook 02 â€“ FAISS index (cleaned) with MiniLM
 2 files changed, 492 insertions(+)
 create mode 100644 data/processed/faiss_hci_small.dvc
 create mode 100644 notebooks/02_ingest_pdfs.ipynb


[WARNING] Unstaged files detected.
[INFO] Stashing unstaged files to C:\Users\Akhan\.cache\pre-commit\patch1746520025-13576.
black................................................(no files to check)Skipped
ruff.................................................(no files to check)Skipped
fix end of files.........................................................Passed
trim trailing whitespace.................................................Passed
[INFO] Restored changes from C:\Users\Akhan\.cache\pre-commit\patch1746520025-13576.


In [19]:
!dvc push


Everything is up to date.


In [20]:
!git push


To github.com:Asadunipr/rag-hci-interaction.git
   60ebf12..8d3ac95  main -> main


In [1]:
vectordb.save_local("../data/processed/faiss_hci_small")


NameError: name 'vectordb' is not defined